# Segmenting and Clustering Neighborhoods in Toronto

Import all libraries

In [84]:
from bs4 import BeautifulSoup
import pandas
import requests

Get the html of the requested page with request

In [85]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

Create an empty dataframe for the final table

In [86]:
postCodes = pandas.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

Extract the wanted information from the html with BeautifulSoup

In [87]:
soup = BeautifulSoup(page.content, 'html.parser')

# Extract the table
postTable = soup.find('table', class_='wikitable sortable')
# Extract all table rows
trs = postTable.find_all('tr')

# Loop over rows without the header
for i in trs[1:]:
    # Extract all row elements
    tds = i.find_all('td')
    elements = []
    for j in tds:
        # Check if there is a hyperlink in the element
        resolve_link = j.find_all('a')
        if len(resolve_link)>0:
            elements.append(resolve_link[0].get_text().rstrip())
        else:
            elements.append(j.get_text().rstrip())
    
    # Skip row if Borough not assigned
    if elements[1] == "Not assigned":
        continue
    # Copy Borough name to Neighborhood if Neighborhood not assigned
    if elements[2] == "Not assigned":
        elements[2] = elements[1]
        
    # Append the table row to the dataframe
    postCodes.loc[len(postCodes) + 1] = elements


Now aggregate the dataframe by grouping of the postal codes and join the Neighborhood by comma

In [88]:
postCodes = postCodes.groupby(["PostalCode"]).agg({"Borough": "first", "Neighborhood": ", ".join}).reset_index()

In [89]:
print("Final table has shape: ", postCodes.shape)

Final table has shape:  (103, 3)
